<a href="https://colab.research.google.com/github/nortonvanz/PySpark-Basics/blob/main/notebooks/Case_3_PySpark_Exercices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
#!tar xf spark-3.1.2-bin-hadoop2.7.tgz
#!pip install -q findspark
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.4 MB/s eta 0:00:00


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()


In [170]:
#imports
import random
from pyspark.sql.window import Window
from unidecode import unidecode

# Load Datasets

In [81]:
#Carrega datasets do github:
!wget --quiet --show-progress https://raw.githubusercontent.com/iara/desafio_cogna/main/dataset/tabela_profile.csv
!wget --quiet --show-progress https://raw.githubusercontent.com/iara/desafio_cogna/main/dataset/tabela_carro.csv
!wget --quiet --show-progress https://raw.githubusercontent.com/iara/desafio_cogna/main/dataset/tabela_emprego.csv

tabela_profile.csv. 100%[===================>]   1.30M  --.-KB/s    in 0.07s   
tabela_carro.csv.1  100%[===================>] 159.33K  --.-KB/s    in 0.04s   
tabela_emprego.csv. 100%[===================>] 196.54K  --.-KB/s    in 0.04s   


In [82]:
#arquivos salvos no Colab
!ls

sample_data		       tabela_carro.csv    tabela_emprego.csv.1
spark-3.1.2-bin-hadoop2.7      tabela_carro.csv.1  tabela_profile.csv
spark-3.1.2-bin-hadoop2.7.tgz  tabela_emprego.csv  tabela_profile.csv.1


In [83]:
df_pessoa = spark.read.csv("tabela_profile.csv", sep=",", inferSchema=True, header=True)
df_carro = spark.read.csv("tabela_carro.csv", sep=",", inferSchema=True, header=True)
df_emprego = spark.read.csv("tabela_emprego.csv", inferSchema=True, header=True)

In [84]:
df_emprego.printSchema()

root
 |-- id_profile: integer (nullable = true)
 |-- cargo: string (nullable = true)
 |-- empresa: string (nullable = true)
 |-- id_emprego: integer (nullable = true)



In [85]:
df_pessoa.show(3)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+
|    579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04|
|    739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17|
|     75467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+
only showing top 3 rows



In [86]:
df_carro.show(3)

+----------+-----------+--------------+--------+
|id_profile|placa_carro|data_aquisicao|id_carro|
+----------+-----------+--------------+--------+
|    591942|   ZHT-2654|    2001-03-06|   73634|
|    381762|   LEL-7635|    1999-01-29|   39149|
|    653260|   KUL-6169|    1999-06-13|   48149|
+----------+-----------+--------------+--------+
only showing top 3 rows



In [87]:
df_emprego.show(3)

+----------+------------------+------------------+----------+
|id_profile|             cargo|           empresa|id_emprego|
+----------+------------------+------------------+----------+
|     33062|Engenheiro químico|      da Conceição|     38503|
|     97704|          Contador|Sales Caldeira S/A|     79501|
|     19073|          Marechal|            Fogaça|     33353|
+----------+------------------+------------------+----------+
only showing top 3 rows



# Questões de Negócio

## 1 Quantas porcento das pessoas  empregadas?

In [88]:
#juntar df_pessoa com df_emprego pelo id_profile
df_pessoa = df_pessoa.join(df_emprego, ["id_profile"], "left")

In [89]:
df_pessoa.show(5)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|cargo|empresa|id_emprego|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|    579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04| null|   null|      null|
|    739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17| null|   null|      null|
|     75467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25| null|   null|      null|
|    233935|  helenaalves|Dr. João Pedro Fe...|   H|Loteamento Barbos...|mariamelo@hotmail...| 1939-02-04| null|   null|      null|
|     42638|  alexandre30|     Juliana Peixoto|   M|Rua de Castro  41...|   

In [90]:
df_pessoa.createOrReplaceTempView("PESSOA")

In [91]:
#SPARK SQL
spark.sql('''
          SELECT
            CASE WHEN CARGO IS NULL THEN 'NÃO' ELSE 'SIM' END AS EMPREGADO,
            COUNT(ID_PROFILE) AS PESSOAS
          FROM PESSOA
          GROUP BY EMPREGADO

        ''').show(5)

+---------+-------+
|EMPREGADO|PESSOAS|
+---------+-------+
|      NÃO|   6485|
|      SIM|   3521|
+---------+-------+



In [92]:
# % de empregados:
3521 / 10006 * 100

35.188886667999206

In [93]:
# SQL FUNCTION

#criar campo tem_emprego, com 0 ou 1:
df_pessoa2 = df_pessoa.withColumn("tem_emprego", when(col("id_emprego").isNotNull(), 1).when(col("id_emprego").isNull(), 0))

In [94]:
df_pessoa2.show(3)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|cargo|empresa|id_emprego|tem_emprego|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----------+
|    579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04| null|   null|      null|          0|
|    739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17| null|   null|      null|          0|
|     75467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25| null|   null|      null|          0|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----

In [95]:
tem_emprego = df_pessoa2.where(col("tem_emprego") == 1).count()
total_pessoas = df_pessoa2.count()

porcentagem=tem_emprego/total_pessoas*100

print(f"tem_emprego={tem_emprego}")
print(f"total_pessoas={total_pessoas}")
print(f"quem tem emprego={porcentagem}%")

tem_emprego=3521
total_pessoas=10006
quem tem emprego=35.188886667999206%


## 2 Criar os salarios anuais dos empregados de forma randomica, sendo que o minimo é 1000

In [96]:
@udf
def gera_salarios():
  return random.randint(1000, 1000000)

df_emprego = df_emprego.withColumn("salario", gera_salarios())

In [97]:
df_emprego.show(5)

+----------+--------------------+------------------+----------+-------+
|id_profile|               cargo|           empresa|id_emprego|salario|
+----------+--------------------+------------------+----------+-------+
|     33062|  Engenheiro químico|      da Conceição|     38503|  19067|
|     97704|            Contador|Sales Caldeira S/A|     79501| 394583|
|     19073|            Marechal|            Fogaça|     33353| 908783|
|     18918|               Cumim|            da Luz|     24143| 461566|
|     61048|Conferente de exp...|        Cavalcanti|     86763| 190412|
+----------+--------------------+------------------+----------+-------+
only showing top 5 rows



## 3 Qual a média salarial de cada cargo?

In [98]:
df_emprego.show(10)

+----------+--------------------+--------------------+----------+-------+
|id_profile|               cargo|             empresa|id_emprego|salario|
+----------+--------------------+--------------------+----------+-------+
|     33062|  Engenheiro químico|        da Conceição|     38503| 529983|
|     97704|            Contador|  Sales Caldeira S/A|     79501| 788754|
|     19073|            Marechal|              Fogaça|     33353| 197971|
|     18918|               Cumim|              da Luz|     24143|  53489|
|     61048|Conferente de exp...|          Cavalcanti|     86763| 164434|
|     75089|Técnico em docume...| Moraes Moreira S.A.|     35470| 898145|
|     11338| Polidor de produção|               Pinto|     48914| 929446|
|     53918|        Carnavalesco|        da Rosa - ME|     71077| 702198|
|     11512|          Cartunista|Nunes Moreira e F...|     28114|  79889|
|     29897|Agente penitenciário|              Aragão|     12559| 282171|
+----------+--------------------+-----

In [99]:
#SQL FUNCTIONS

#para um cargo:
df_emprego.filter(col("cargo") == "Engenheiro químico").show()

+----------+------------------+------------------+----------+-------+
|id_profile|             cargo|           empresa|id_emprego|salario|
+----------+------------------+------------------+----------+-------+
|     33062|Engenheiro químico|      da Conceição|     38503| 824360|
|     57836|Engenheiro químico|Barbosa Gomes - ME|     90544| 701524|
|     55758|Engenheiro químico|              Dias|     90896| 976556|
|     59095|Engenheiro químico|             Alves|     47752| 613914|
|    806335|Engenheiro químico|          Silveira|     30103| 882003|
|    698440|Engenheiro químico|             Lopes|     10085| 120233|
|    542447|Engenheiro químico|       Costela S/A|      8995| 456550|
|    403934|Engenheiro químico|           Rezende|     83488| 850775|
|    230830|Engenheiro químico|            Vieira|     64458| 935184|
|    827496|Engenheiro químico|             Gomes|     44632| 590958|
|    216314|Engenheiro químico|           Barbosa|     14463| 988135|
|    205086|Engenhei

In [100]:
# Média salarial por cargo:
df_emprego.groupBy("cargo").agg(avg("salario").alias("media_salarial")).show(10)

+--------------------+------------------+
|               cargo|    media_salarial|
+--------------------+------------------+
|Engenheiro de erg...|          468654.5|
|Auxiliar de repro...| 476013.6666666667|
|        Neurologista|524205.14285714284|
|             Tenente|          486459.0|
|          Gastrônomo|470434.14285714284|
|           Bailarina| 558706.8571428572|
|Profissional de r...| 388652.0909090909|
|         Pastilheiro| 660507.8181818182|
|             Lavador|          506298.8|
|Classificador con...|          518670.0|
+--------------------+------------------+
only showing top 10 rows



In [101]:
#SPARK SQL
df_emprego.createOrReplaceTempView("EMPREGO")

In [102]:
#SPARK SQL
spark.sql('''
          SELECT
            CARGO,
            AVG(SALARIO) AS MEDIA_SALARIAL
          FROM EMPREGO
          WHERE CARGO = 'Tenente'
          GROUP BY CARGO

        ''').show(100)

+-------+--------------+
|  CARGO|MEDIA_SALARIAL|
+-------+--------------+
|Tenente|     502384.25|
+-------+--------------+



In [103]:
#PS: alterando o valor devido ao random, mas a lógica acima é aplicável em um dataset real.

## 4 Gerar um numero sequencial para cada cargo, e eliminar todos as linhas maiores que 1.

In [104]:
df_emprego.show(3)

+----------+------------------+------------------+----------+-------+
|id_profile|             cargo|           empresa|id_emprego|salario|
+----------+------------------+------------------+----------+-------+
|     33062|Engenheiro químico|      da Conceição|     38503| 577637|
|     97704|          Contador|Sales Caldeira S/A|     79501| 479217|
|     19073|          Marechal|            Fogaça|     33353| 177941|
+----------+------------------+------------------+----------+-------+
only showing top 3 rows



In [105]:
windows = Window.partitionBy("cargo").orderBy(col("salario").asc())

In [106]:
#cria rownum para cada cargo
df_emprego = df_emprego.withColumn("row_number", row_number().over(windows))

In [107]:
df_emprego.show(5)

+----------+--------------------+-------------------+----------+-------+----------+
|id_profile|               cargo|            empresa|id_emprego|salario|row_number|
+----------+--------------------+-------------------+----------+-------+----------+
|     98577|Auxiliar de repro...|  Duarte Jesus - ME|     95203| 332350|         1|
|     40630|Auxiliar de repro...|              Nunes|     31885| 727394|         2|
|      3331|Auxiliar de repro...|   Gomes Rocha - EI|     30316| 779512|         3|
|     15797|Engenheiro de erg...|Azevedo da Luz S.A.|     14731| 120214|         1|
|     70290|Engenheiro de erg...|             Moraes|     33119| 137431|         2|
+----------+--------------------+-------------------+----------+-------+----------+
only showing top 5 rows



In [108]:
#rownum se usa muito pra eliminar duplicados na prática

In [109]:
#manter só rownum = 1
df_emprego = df_emprego.filter(col("row_number") == 1)

In [125]:
df_emprego.show(5)

+----------+--------------------+--------------------+----------+-------+----------+
|id_profile|               cargo|             empresa|id_emprego|salario|row_number|
+----------+--------------------+--------------------+----------+-------+----------+
|     40630|Auxiliar de repro...|               Nunes|     31885| 840793|         1|
|     26571|Engenheiro de erg...|           Melo - ME|     52767| 115102|         1|
|     58023|          Gastrônomo|Moreira Aragão e ...|     70592| 235869|         1|
|     76400|        Neurologista|          da Paz S/A|     65881| 127962|         1|
|     98111|             Tenente|         Duarte - ME|     59388| 103633|         1|
+----------+--------------------+--------------------+----------+-------+----------+
only showing top 5 rows



In [128]:
#SPARK SQL

# Executa a query SQL para adicionar a coluna de número sequencial
spark.sql('''
         SELECT *,
            ROW_NUMBER() OVER (PARTITION BY cargo ORDER BY id_profile) as numero_sequencial
        FROM EMPREGO

        ''').show(10)

+----------+--------------------+--------------------+----------+-------+----------+-----------------+
|id_profile|               cargo|             empresa|id_emprego|salario|row_number|numero_sequencial|
+----------+--------------------+--------------------+----------+-------+----------+-----------------+
|      3331|Auxiliar de repro...|    Gomes Rocha - EI|     30316|  55024|         1|                1|
|     26571|Engenheiro de erg...|           Melo - ME|     52767| 105331|         1|                1|
|    887493|          Gastrônomo|            Nogueira|     13844| 102513|         1|                1|
|    906712|        Neurologista|               Cunha|     61932| 196308|         1|                1|
|     79273|             Tenente|            Caldeira|     70732| 107532|         1|                1|
|      4409|           Bailarina|        da Rosa - EI|     94208| 151847|         1|                1|
|    187034|Classificador con...|Araújo da Cunha - ME|     96454| 200868|

In [132]:
#df_emprego atualizada
df_emprego.show(5)

+----------+--------------------+--------------------+----------+-------+----------+
|id_profile|               cargo|             empresa|id_emprego|salario|row_number|
+----------+--------------------+--------------------+----------+-------+----------+
|     98577|Auxiliar de repro...|   Duarte Jesus - ME|     95203| 267108|         1|
|     57384|Engenheiro de erg...|             Peixoto|       135| 107557|         1|
|    696563|          Gastrônomo|Teixeira Caldeira...|     52039| 195516|         1|
|    210127|        Neurologista|          Melo Ltda.|     34664| 176028|         1|
|    197143|             Tenente|         Moraes S.A.|     32418| 142557|         1|
+----------+--------------------+--------------------+----------+-------+----------+
only showing top 5 rows



In [133]:
#SPARK SQL
#recriar view EMPREGO com os novos dados:
df_emprego.createOrReplaceTempView("EMPREGO")

In [134]:
spark.sql('''
         SELECT *
        FROM EMPREGO
        ''').show(10)

+----------+--------------------+--------------------+----------+-------+----------+
|id_profile|               cargo|             empresa|id_emprego|salario|row_number|
+----------+--------------------+--------------------+----------+-------+----------+
|     40630|Auxiliar de repro...|               Nunes|     31885|  16248|         1|
|    414875|Engenheiro de erg...|   da Paz Rocha - ME|     22937| 344557|         1|
|     65152|          Gastrônomo|     Santos e Filhos|     14812| 155815|         1|
|    169437|        Neurologista|   Silva Cardoso S/A|     81789| 148930|         1|
|    395417|             Tenente|  Farias Cunha Ltda.|     94370| 295823|         1|
|    672344|           Bailarina|Lima Nascimento - EI|     71563| 197964|         1|
|    220833|Classificador con...|           Pires S/A|      8869| 206060|         1|
|    215500|             Lavador|             Rezende|     90093| 309444|         1|
|    899315|         Pastilheiro|              Araújo|     58340|

## 5 Quem é a pessoa mais jovem a ter um carro?

In [138]:
df_pessoa.show(3)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|cargo|empresa|id_emprego|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|    579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04| null|   null|      null|
|    739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17| null|   null|      null|
|     75467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25| null|   null|      null|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
only showing top 3 rows



In [137]:
df_carro.show(3)

+----------+-----------+--------------+--------+
|id_profile|placa_carro|data_aquisicao|id_carro|
+----------+-----------+--------------+--------+
|    591942|   ZHT-2654|    2001-03-06|   73634|
|    381762|   LEL-7635|    1999-01-29|   39149|
|    653260|   KUL-6169|    1999-06-13|   48149|
+----------+-----------+--------------+--------+
only showing top 3 rows



In [ ]:
# SPARK SQL:

In [142]:
#temos a view de pessoa já, criar a de carro:
df_carro.createOrReplaceTempView("CARRO")

In [154]:
spark.sql('''
          SELECT
            P.ID_PROFILE,
            P.ANIVERSARIO
          FROM CARRO C
          INNER JOIN PESSOA P ON P.id_profile = C.id_profile
          ORDER BY P.ANIVERSARIO DESC
          LIMIT 1
        ''').show(10)

+----------+-----------+
|ID_PROFILE|ANIVERSARIO|
+----------+-----------+
|    875425| 2002-12-09|
+----------+-----------+



In [ ]:
# SQL FUNCIONS

In [155]:
df_carro2 = df_pessoa.join(df_carro, ["id_profile"], "inner")

In [156]:
df_carro2.withColumn("aniversario", to_date(col("aniversario"),"yyyy-MM-dd"))
df_carro2.orderBy(col("aniversario").desc()).show(1)

+----------+-----------+-------------+----+--------------------+-------------------+-----------+-----+-------+----------+-----------+--------------+--------+
|id_profile|   username|         name|sexo|            endereço|              email|aniversario|cargo|empresa|id_emprego|placa_carro|data_aquisicao|id_carro|
+----------+-----------+-------------+----+--------------------+-------------------+-----------+-----+-------+----------+-----------+--------------+--------+
|    875425|castrorenan|Rafaela Viana|   M|Avenida Freitas  ...|ocunha@yahoo.com.br| 2002-12-09| null|   null|      null|   DAY-1821|    2014-05-25|   75332|
+----------+-----------+-------------+----+--------------------+-------------------+-----------+-----+-------+----------+-----------+--------------+--------+
only showing top 1 row



In [157]:
# por sexo:
df_carro2.groupBy("sexo").agg(max(col("aniversario"))).show()

+----+----------------+
|sexo|max(aniversario)|
+----+----------------+
|   M|      2002-12-09|
|   H|      2002-12-07|
+----+----------------+



## 6 Normalizar o id_profile para 8 digitos nos dataframes

In [158]:
df_emprego.show(3)

+----------+--------------------+----------------+----------+-------+----------+
|id_profile|               cargo|         empresa|id_emprego|salario|row_number|
+----------+--------------------+----------------+----------+-------+----------+
|      3331|Auxiliar de repro...|Gomes Rocha - EI|     30316| 599931|         1|
|    327315|Engenheiro de erg...|           Cunha|     61308| 232169|         1|
|     65152|          Gastrônomo| Santos e Filhos|     14812| 348563|         1|
+----------+--------------------+----------------+----------+-------+----------+
only showing top 3 rows



In [160]:
#usar função lbad para preencher com zero:
df_emprego = df_emprego.withColumn("id_profile", lpad(col("id_profile"),8, '0'))
df_carro = df_carro.withColumn("id_profile", lpad(col("id_profile"),8, '0'))
df_pessoa = df_pessoa.withColumn("id_profile", lpad(col("id_profile"),8, '0'))

In [163]:
df_pessoa.show(5)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|cargo|empresa|id_emprego|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|  00579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04| null|   null|      null|
|  00739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17| null|   null|      null|
|  00075467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25| null|   null|      null|
|  00233935|  helenaalves|Dr. João Pedro Fe...|   H|Loteamento Barbos...|mariamelo@hotmail...| 1939-02-04| null|   null|      null|
|  00042638|  alexandre30|     Juliana Peixoto|   M|Rua de Castro  41...|   

## 7 Remover o acento dos nomes no dataframe de pessoa.

In [175]:
#importada lib: unidecode
print(unidecode('chapéu, cão'))

chapeu, cao


In [180]:
#name original
df_pessoa.show(5)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|cargo|empresa|id_emprego|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|  00579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04| null|   null|      null|
|  00739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17| null|   null|      null|
|  00075467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25| null|   null|      null|
|  00233935|  helenaalves|Dr. João Pedro Fe...|   H|Loteamento Barbos...|mariamelo@hotmail...| 1939-02-04| null|   null|      null|
|  00042638|  alexandre30|     Juliana Peixoto|   M|Rua de Castro  41...|   

In [185]:
#funçõa recebe o nome, e retorna sem acento:
@udf
def unicode_name(name):
  return unidecode(name)

In [191]:
df_pessoa = df_pessoa.withColumn("name", unicode_name(col("name")))

In [192]:
#coluna nome tratada:
df_pessoa.select("name").show(5)

+--------------------+
|                name|
+--------------------+
|    Augusto Ferreira|
|        Helena Viana|
|Srta. Amanda Teix...|
|Dr. Joao Pedro Fe...|
|     Juliana Peixoto|
+--------------------+
only showing top 5 rows



In [ ]:
#outra alternativa:

# unicodeUDF = udf(lambda z: unidecode(z))
# df_pessoa.select(unicodeUDF(col("name"))).show()

## 8 Quantas pessoas nasceram em cada década?

In [ ]:
#SPARK SQL

In [193]:
#temos a view de pessoa já, criar a de carro:
df_pessoa.createOrReplaceTempView("PESSOA")

In [250]:
spark.sql('''
          SELECT
              EXTRACT (YEAR FROM ANIVERSARIO) AS ANO_NASCIMENTO,
              CASE
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1910) AND (EXTRACT (YEAR FROM ANIVERSARIO) <1920) THEN 10
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1920) AND (EXTRACT (YEAR FROM ANIVERSARIO) <1930) THEN 20
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1930) AND (EXTRACT (YEAR FROM ANIVERSARIO) <1940) THEN 30
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1940) AND (EXTRACT (YEAR FROM ANIVERSARIO) <1950) THEN 40
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1950) AND (EXTRACT (YEAR FROM ANIVERSARIO) <1960) THEN 50
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1960) AND (EXTRACT (YEAR FROM ANIVERSARIO) <1970) THEN 60
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1970) AND (EXTRACT (YEAR FROM ANIVERSARIO) <1980) THEN 70
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1980) AND (EXTRACT (YEAR FROM ANIVERSARIO) <1990) THEN 80
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 1990) AND (EXTRACT (YEAR FROM ANIVERSARIO) <2000) THEN 90
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 2000) AND (EXTRACT (YEAR FROM ANIVERSARIO) <2010) THEN 2000
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 2010) AND (EXTRACT (YEAR FROM ANIVERSARIO) <2020) THEN 2010
                WHEN (EXTRACT (YEAR FROM ANIVERSARIO) >= 2020) AND (EXTRACT (YEAR FROM ANIVERSARIO) <2030) THEN 2020
                ELSE NULL
              END AS DECADA

          FROM PESSOA

        ''').show(15)

+--------------+------+
|ANO_NASCIMENTO|DECADA|
+--------------+------+
|          1934|    30|
|          2000|  2000|
|          1936|    30|
|          1939|    30|
|          1912|    10|
|          1947|    40|
|          1952|    50|
|          1999|    90|
|          1992|    90|
|          1952|    50|
|          1988|    80|
|          1915|    10|
|          1946|    40|
|          1912|    10|
|          1989|    80|
+--------------+------+
only showing top 15 rows



In [ ]:
#SQL FUNCTIONS

In [251]:
df2 = df_pessoa.groupBy(year(col("aniversario")).alias("ano_nascimento")).agg(count("id_profile").alias("count_nascimento"))

In [252]:
df2.show(3)

+--------------+----------------+
|ano_nascimento|count_nascimento|
+--------------+----------------+
|          1959|              85|
|          1990|             178|
|          1975|              87|
+--------------+----------------+
only showing top 3 rows



In [256]:
df4=df2.withColumn("decada", when((col("ano_nascimento") >=1900) & (col("ano_nascimento") <1910), 00)
                         .when((col("ano_nascimento") >=1910) & (col("ano_nascimento") <1920), 10)
                         .when((col("ano_nascimento") >=1920) & (col("ano_nascimento") <1930), 20)
                         .when((col("ano_nascimento") >=1930) & (col("ano_nascimento") <1940), 30)
                         .when((col("ano_nascimento") >=1940) & (col("ano_nascimento") <1950), 40)
                         .when((col("ano_nascimento") >=1950) & (col("ano_nascimento") <1960), 50)
                         .when((col("ano_nascimento") >=1960) & (col("ano_nascimento") <1970), 60)
                         .when((col("ano_nascimento") >=1970) & (col("ano_nascimento") <1980), 70)
                         .when((col("ano_nascimento") >=1980) & (col("ano_nascimento") <1990), 80)
                         .when((col("ano_nascimento") >=1990) & (col("ano_nascimento") <2000), 90)
                         .when((col("ano_nascimento") >=2000) & (col("ano_nascimento") <2010), 2000)
                         .when((col("ano_nascimento") >=2010) & (col("ano_nascimento") <2020), 2010)
                         .otherwise(3333)
              )

In [257]:
df5=df4.groupBy(col("decada")).agg(sum(col("count_nascimento").alias("count_nascimento_decada")))

In [255]:
df4.filter(col("ano_nascimento")> 0).show()

+--------------+----------------+------+
|ano_nascimento|count_nascimento|decada|
+--------------+----------------+------+
|          1959|              85|    50|
|          1990|             178|    90|
|          1975|              87|    70|
|          1977|              94|    70|
|          1924|              94|    20|
|          1974|              89|    70|
|          1927|              98|    20|
|          1955|              92|    50|
|          1978|              80|    70|
|          1925|              92|    20|
|          1908|              94|     0|
|          1961|              75|    60|
|          1942|              93|    40|
|          1939|              74|    30|
|          1944|              81|    40|
|          1922|              73|    20|
|          1952|              95|    50|
|          1934|              96|    30|
|          1956|              91|    50|
|          1988|              79|    80|
+--------------+----------------+------+
only showing top

## 9 Qual a década que que há mais pessoas empregadas?


In [267]:
#criar campo tem_emprego, com 0 ou 1:
df_pessoa2 = df_pessoa.withColumn("tem_emprego", when(col("id_emprego").isNotNull(), 1).when(col("id_emprego").isNull(), 0))

In [268]:
df_pessoa2.show(3)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|cargo|empresa|id_emprego|tem_emprego|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----------+
|  00579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04| null|   null|      null|          0|
|  00739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17| null|   null|      null|          0|
|  00075467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25| null|   null|      null|          0|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----

In [270]:
#df_pessoa2 =
df_pessoa2.filter((col("tem_emprego") == 1) & (col("aniversario").isNotNull())).show(3)

+----------+-------------------+----------------+----+--------------------+--------------------+-----------+-----------------+-----------------+----------+-----------+
|id_profile|           username|            name|sexo|            endereço|               email|aniversario|            cargo|          empresa|id_emprego|tem_emprego|
+----------+-------------------+----------------+----+--------------------+--------------------+-----------+-----------------+-----------------+----------+-----------+
|  00023737|      das-nevesluna|Maria Julia Dias|   M|Núcleo Kamilly Pi...|enzo-gabriel79@ig...| 1945-10-28|          Armador|    da Cruz Ltda.|     41333|          1|
|  00523014|da-mataluiz-gustavo|    Joao Cardoso|   H|Jardim Rezende Ca...|santoslais@uol.co...| 1913-12-02|    Ferramenteiro|da Paz Cunha - ME|     45752|          1|
|  00203338|            gda-paz|    Noah da Mota|   H|Viaduto de Jesus ...|danilonogueira@gm...| 1961-03-08|Agente de viagens|         Nogueira|     84503|     

In [271]:
df_pessoa2 = df_pessoa2.groupBy(year(col("aniversario")).alias("ano_nascimento")).agg(count("id_profile").alias("count_nascimento"))
df_pessoa2 = df_pessoa2.filter(col("ano_nascimento").isNotNull())
df_pessoa2 = df_pessoa2.orderBy(col("ano_nascimento").asc())


df_pessoa2 = df_pessoa2.withColumn("decada", when((col("ano_nascimento") >=1900) & (col("ano_nascimento") <1910), 00)
                         .when((col("ano_nascimento") >=1910) & (col("ano_nascimento") <1920), 10)
                         .when((col("ano_nascimento") >=1920) & (col("ano_nascimento") <1930), 20)
                         .when((col("ano_nascimento") >=1930) & (col("ano_nascimento") <1940), 30)
                         .when((col("ano_nascimento") >=1940) & (col("ano_nascimento") <1950), 40)
                         .when((col("ano_nascimento") >=1950) & (col("ano_nascimento") <1960), 50)
                         .when((col("ano_nascimento") >=1960) & (col("ano_nascimento") <1970), 60)
                         .when((col("ano_nascimento") >=1970) & (col("ano_nascimento") <1980), 70)
                         .when((col("ano_nascimento") >=1980) & (col("ano_nascimento") <1990), 80)
                         .when((col("ano_nascimento") >=1990) & (col("ano_nascimento") <2000), 90)
                         .when((col("ano_nascimento") >=2000) & (col("ano_nascimento") <2010), 200)
                         .when((col("ano_nascimento") >=2010) & (col("ano_nascimento") <2020), 210)
                         .otherwise(3333)
              )

In [272]:
df_pessoa2=df_pessoa2.groupBy(col("decada")).agg(sum(col("count_nascimento")).alias("count_nascimento_decada"))

In [275]:
#na década de 90:
df_pessoa2.orderBy(col("count_nascimento_decada").desc()).show(1)

+------+-----------------------+
|decada|count_nascimento_decada|
+------+-----------------------+
|    90|                   2417|
+------+-----------------------+
only showing top 1 row

